In [ ]:
%pip install pandas

In [3]:
# ===============================================================
# Classificação Binária – Iris (todas as 4 features)
# Versicolor (1, classe positiva) vs Virginica (0, classe negativa)
# Modelo: Regressão Logística (com StandardScaler)
# Saídas:
#   - visão do dataset + 10 primeiras linhas
#   - métricas (acc/prec/recall/F1) e matriz de confusão no TESTE
#   - predições individuais com probabilidades
#   - interpretação via coeficientes da RL
# ===============================================================

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, classification_report
)

RANDOM_STATE = 42
TEST_SIZE = 0.30

# ---------------------------------------------------------------
# 1) Carregar Iris e tornar BINÁRIA: 1=versicolor, 0=virginica
# ---------------------------------------------------------------
iris = load_iris()
X_full = iris.data            # (150, 4): [sepal len, sepal wid, petal len, petal wid]
y_full = iris.target          # 0=setosa, 1=versicolor, 2=virginica

# Filtrar: remover setosa (0). Ficam rótulos 1 e 2 → mapear para {1:1, 2:0}
mask = (y_full != 0)
X = X_full[mask]
y = (y_full[mask] == 1).astype(int)  # versicolor → 1 (positiva), virginica → 0

feature_names = iris.feature_names  # usaremos TODAS as 4 features
class_map = {0: "virginica (negativa)", 1: "versicolor (positiva)"}

print("=== Dataset: Iris (binário) ===")
print("Classes: 0 = virginica (negativa), 1 = versicolor (positiva)")
print("Features:", list(feature_names))
print(f"Instâncias totais: {X.shape[0]} | Nº de features: {X.shape[1]}")

# Mostrar as 10 primeiras linhas (features)
df = pd.DataFrame(X, columns=feature_names)
df["classe"] = [class_map[c] for c in y]
print("\n=== 10 primeiras linhas ===")
print(df.head(10).to_string(index=False))

# Distribuição global das classes
vals, cnts = np.unique(y, return_counts=True)
print("\nDistribuição por classe (total):")
for v, c in zip(vals, cnts):
    print(f"  classe {v} = {class_map[v]:<22s} -> {c}")

# ---------------------------------------------------------------
# 2) Split treino/teste (estratificado preserva proporções)
# ---------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)
print(f"\nTamanhos -> treino: {X_train.shape[0]} | teste: {X_test.shape[0]}")

# ---------------------------------------------------------------
# 3) Pipeline: padronização + Regressão Logística
# ---------------------------------------------------------------
# Por que padronizar?
# - A RL otimiza uma função que converge melhor quando as features têm escala similar.
# - Coeficientes ficam em escalas comparáveis (ajuda na interpretação relativa).
modelo = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))
])

# ---------------------------------------------------------------
# 4) Treinar
# ---------------------------------------------------------------
modelo.fit(X_train, y_train)

# ---------------------------------------------------------------
# 5) Avaliar no TESTE
# ---------------------------------------------------------------
y_pred = modelo.predict(X_test)
cm = confusion_matrix(y_test, y_pred)  # linhas = Real, colunas = Previsto

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, pos_label=1)  # positiva = versicolor
rec  = recall_score(y_test, y_pred, pos_label=1)
f1   = f1_score(y_test, y_pred, pos_label=1)

print("\n=== Desempenho no TESTE ===")
print(f"Acurácia: {acc:.3f}")
print(f"Precisão (positiva=versicolor): {prec:.3f}")
print(f"Recall   (positiva=versicolor): {rec:.3f}")
print(f"F1       (positiva=versicolor): {f1:.3f}")

print("\nMatriz de Confusão (linhas=Real, colunas=Previsto):")
print("           Prev 0  Prev 1")
print(f"Real 0  |   {cm[0,0]:>3}     {cm[0,1]:>3}   <- virginica (0)")
print(f"Real 1  |   {cm[1,0]:>3}     {cm[1,1]:>3}   <- versicolor (1)")

print("\nRelatório por classe:")
print(classification_report(y_test, y_pred, target_names=["virginica", "versicolor"]))

# ---------------------------------------------------------------
# 6) Predições individuais (com 4 features)
# ---------------------------------------------------------------
# Ordem das features: [sepal length, sepal width, petal length, petal width]
amostras = np.array([
    [5.8, 2.8, 4.6, 1.4],  # geralmente mais versicolor
    [6.3, 3.0, 5.8, 2.2],  # geralmente mais virginica
    [6.0, 2.9, 4.5, 1.5]   # ponto “fronteira” para ver probabilidade
])

probas = modelo.predict_proba(amostras)  # colunas: [P(0), P(1)]
preds  = modelo.predict(amostras)

print("=== Predições Individuais ===")
for x, p, pred in zip(amostras, probas, preds):
    print(f"Entrada {list(x)} -> prev={'versicolor (1)' if pred==1 else 'virginica (0)'} "
          f"| P(virginica)= {p[0]:.3f} | P(versicolor)= {p[1]:.3f}")


=== Dataset: Iris (binário) ===
Classes: 0 = virginica (negativa), 1 = versicolor (positiva)
Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Instâncias totais: 100 | Nº de features: 4

=== 10 primeiras linhas ===
 sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)                classe
               7.0               3.2                4.7               1.4 versicolor (positiva)
               6.4               3.2                4.5               1.5 versicolor (positiva)
               6.9               3.1                4.9               1.5 versicolor (positiva)
               5.5               2.3                4.0               1.3 versicolor (positiva)
               6.5               2.8                4.6               1.5 versicolor (positiva)
               5.7               2.8                4.5               1.3 versicolor (positiva)
               6.3               3.3                4.7               